<a href="https://colab.research.google.com/github/nobertomaciel/PLN-ANIMA/blob/main/PLN_Analise_de_sentimentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gensim
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 17.6 MB/s eta 0:00:00


In [10]:
# ===============================================
# Exemplo completo de PLN com Classificação de Sentimentos
# ===============================================

import re
import nltk
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import string

In [11]:
# ===============================================
# 1. Recursos do NLTK
# ===============================================
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
# ===============================================
# 2. Dataset realista de avaliações
# ===============================================
# 1 = positivo, 0 = negativo
dados = {
    "avaliacao": [
        "O celular é ótimo, tem uma câmera excelente!",
        "Horrível! Que produto terrível, não recomendo a ninguém.",
        "Entrega rápida e produto de boa qualidade.",
        "A bateria dura pouco e esquenta muito. Péssimo!",
        "Muito bom, atendeu todas as minhas expectativas.",
        "Não gostei, o material parece frágil e a tela risca fácil.",
        "Superou minhas expectativas, recomendo demais!",
        "O som é baixo e a bateria acabou em poucas horas.",
        "Produto maravilhoso, desempenho excelente.",
        "Defeituoso, não funcionou desde o primeiro dia."
    ],
    "sentimento": [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
}

df = pd.DataFrame(dados)

In [13]:
# ===============================================
# 3. Função de pré-processamento
# ===============================================
def preprocessar_texto(texto):
    texto = texto.lower()
    texto = texto.replace('\n', ' ')
    texto = re.sub(r'[^a-zA-Zà-úÀ-Ú\s]', '', texto)
    tokens = nltk.word_tokenize(texto, language='portuguese')
    tokens = [t for t in tokens if t not in string.punctuation]
    stop_words = set(stopwords.words('portuguese'))
    tokens = [t for t in tokens if t not in stop_words and len(t) > 2]
    return " ".join(tokens)

df["texto_limpo"] = df["avaliacao"].apply(preprocessar_texto)

In [ ]:
# ===============================================
# 4. Representações vetoriais (Bag-of-Words e TF-IDF)
# ===============================================
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["texto_limpo"])
y = df["sentimento"]

In [ ]:
# ===============================================
# 5. Divisão de dados (treino e teste)
# ===============================================
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# ===============================================
# 6. Aplicação do Machine Learning: modelo de classificação (Regressão Logística)
# ===============================================
modelo = LogisticRegression(max_iter=1000)
modelo.fit(X_train, y_train)

In [ ]:
# ===============================================
# 7. Avaliação do modelo
# ===============================================
y_pred = modelo.predict(X_test)
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=["Negativo", "Positivo"]))

print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
# ===============================================
# 8. Teste com novas avaliações
# ===============================================
novas_avaliacoes = [
    "A câmera é incrível e a bateria dura o dia todo.",
    "O produto veio quebrado e não liga mais.",
    "Entrega rápida, mas o desempenho é ruim.",
    "Excelente custo-benefício, estou muito satisfeito!"
]

# Pré-processar e transformar
novas_proc = [preprocessar_texto(a) for a in novas_avaliacoes]
X_novas = vectorizer.transform(novas_proc)
predicoes = modelo.predict(X_novas)

# Exibir resultados
print("\nResultados em novas avaliações:")
for texto, pred in zip(novas_avaliacoes, predicoes):
    sentimento = "Positivo 😀" if pred == 1 else "Negativo 😞"
    print(f"- {texto} → {sentimento}")

In [ ]:
# ===============================================
# 9. Word2Vec (para análise semântica complementar)
# ===============================================
tokens = [t.split() for t in df["texto_limpo"]]
modelo_w2v = Word2Vec(sentences=tokens, vector_size=50, window=3, min_count=1, sg=1)

print("\nPalavras mais similares a 'ótimo':")
try:
    print(modelo_w2v.wv.most_similar('ótimo'))
except KeyError:
    print("A palavra 'ótimo' não está no vocabulário.")